In [1]:
%config InlineBackend.figure_format = 'retina'
%matplotlib inline
import os
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from tqdm import tqdm
sns.set_palette('Dark2')
sns.set_style({'axes.axisbelow': True, 'axes.edgecolor': '.15', 'axes.facecolor': 'white',
               'axes.grid': True, 'axes.labelcolor': '.15', 'axes.linewidth': 1.25, 
               'figure.facecolor': 'white', 'font.family': ['sans-serif'], 'grid.color': '.15',
               'grid.linestyle': ':', 'grid.alpha': .5, 'image.cmap': 'Greys', 
               'legend.frameon': False, 'legend.numpoints': 1, 'legend.scatterpoints': 1,
               'lines.solid_capstyle': 'round', 'axes.spines.right': False, 'axes.spines.top': False,  
               'text.color': '.15',  'xtick.top': False, 'ytick.right': False, 'xtick.color': '.15',
               'xtick.direction': 'out', 'xtick.major.size': 6, 'xtick.minor.size': 3,
               'ytick.color': '.15', 'ytick.direction': 'out', 'ytick.major.size': 6,'ytick.minor.size': 3})
sns.set_context('paper')

#http://phyletica.org/matplotlib-fonts/
import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42

In [2]:
FILENAME = '../data/raw/reinke-science-2013-table-s2.xlsx'

In [3]:
import pathlib

OUTPUT_DIR = pathlib.Path('../output')
if not OUTPUT_DIR.exists():
    os.makedirs(OUTPUT_DIR)

In [4]:
dfs = {}

with pd.ExcelFile(FILENAME) as file_xlsx:
    sheet_names = file_xlsx.sheet_names
    
    for i, sheet_name in enumerate(sheet_names):
        
        if sheet_name.endswith('R2'):
            continue
            
        # The first sheet has caption in first row, which we need to skip
        if i == 0: 
            skiprows = 1
        else:
            skiprows = None
            
        df = file_xlsx.parse(sheet_name, skiprows=skiprows)
        df.columns = ['acceptor_protein'] + list(df.columns)[1:]
        
        df = df.set_index('acceptor_protein')
        df.columns.name = 'donor_protein'
        
        dfs[sheet_name] = df
            


In [5]:
dfs_categorical = {}

for name, df in dfs.items():
    df = dfs[name].copy().replace('<1', 0.5).replace('>5000', 5000.05)
    
    for col in df:
        df[col] = pd.cut(df[col], 
                         [0.0, 1.0, 10, 50, 250, 1000, 5000, 5001], 
                         labels = ['<1', '1-10', '10-50', '50-250', '250-1000', '1000-5000', '>5000'])
        
    dfs_categorical[name] = df
    

# Heatmaps

In [6]:
from scipy.cluster import hierarchy

def cluster_categorical(categorical_df, method='complete', metric='cosine'):
    
    coded_df = categorical_df.copy()
    for col in coded_df.columns:
        coded_df[col] = coded_df[col].cat.codes
        
    max_code = coded_df.max().max()
    
    # NaNs get -1, replace that with max_code
    coded_df = coded_df.replace(-1, max_code)
    
    # We want the code reversed, so highest Kd has fewer numbers:
    coded_df = max_code - coded_df
    
    # cluster rows and cols
    try:
        linkage_rows = hierarchy.linkage(coded_df, method=method, metric=metric, optimal_ordering=True)
    except ValueError:
        # This happens when some rows have distance zero ..
        # Add some randomness
        rs = np.random.RandomState(123)
        linkage_rows = hierarchy.linkage(coded_df + rs.randn(*coded_df.shape), method=method, metric=metric, optimal_ordering=True)
        
    try:
        linkage_cols = hierarchy.linkage(coded_df.T, method=method, metric=metric, optimal_ordering=True)
    except ValueError:
        # This happens when some cols have distance zero ..
        # Add some randomness
        rs = np.random.RandomState(123)
        linkage_cols = hierarchy.linkage((coded_df + rs.randn(*coded_df.shape)).T, method=method, metric=metric, optimal_ordering=True)
        
    
    order_rows = coded_df.index[hierarchy.dendrogram(linkage_rows, no_plot=True)['leaves']]
    order_cols = coded_df.columns[hierarchy.dendrogram(linkage_cols, no_plot=True)['leaves']]
    
    categorical_df = categorical_df.reindex(order_rows)
    categorical_df = categorical_df.reindex(order_cols, axis=1)
    
    return categorical_df
    
    

In [7]:
import palettable

palette = {
    '<1': palettable.cartocolors.sequential.Sunset_7.hex_colors[0], 
    '1-10': palettable.cartocolors.sequential.Sunset_7.hex_colors[1], 
    '10-50': palettable.cartocolors.sequential.Sunset_7.hex_colors[2], 
    '50-250': palettable.cartocolors.sequential.Sunset_7.hex_colors[3], 
    '250-1000': palettable.cartocolors.sequential.Sunset_7.hex_colors[4], 
    '1000-5000': palettable.cartocolors.sequential.Sunset_7.hex_colors[5], 
    '>5000': palettable.cartocolors.sequential.Sunset_7.hex_colors[6],
    'N/A': 'white' 
}

In [8]:
species_dict = {
    'HS': 'Homo sapiens',
    'HSM': "Homo sapiens (manual assay)",
    'CE': 'Caenorhabditis elegans',
    'CEM': 'Caenorhabditis elegans (manual assay)',
    'CI': 'Ciona intestinalis',
    'CS': 'H. sapiens, C. intestinalis (cross-species)',
    'DM': 'Drosophila melanogaster',
    'MB': 'Monosiga brevicollis',
    'NV': 'Nematostella vectensis',
    'SC': 'Saccharomyces cerevisiae',
}
def parse_name(name):
    
    species, temperature, kd = name.split('_')
    
    species_long = species_dict[species]
    
    return f'{species_long} {kd} @ {temperature}°C'

In [9]:
import catheat

In [10]:
[parse_name(k) for k in dfs]

['Homo sapiens Kd @ 37°C',
 'Homo sapiens Kd @ 21°C',
 'Homo sapiens Kd @ 4°C',
 'Homo sapiens (manual assay) Kd @ 37°C',
 'Homo sapiens (manual assay) Kd @ 21°C',
 'Homo sapiens (manual assay) Kd @ 4°C',
 'Ciona intestinalis Kd @ 37°C',
 'Ciona intestinalis Kd @ 21°C',
 'Ciona intestinalis Kd @ 4°C',
 'Drosophila melanogaster Kd @ 37°C',
 'Drosophila melanogaster Kd @ 21°C',
 'Drosophila melanogaster Kd @ 4°C',
 'Caenorhabditis elegans Kd @ 37°C',
 'Caenorhabditis elegans Kd @ 21°C',
 'Caenorhabditis elegans Kd @ 4°C',
 'Caenorhabditis elegans (manual assay) Kd @ 37°C',
 'Caenorhabditis elegans (manual assay) Kd @ 21°C',
 'Caenorhabditis elegans (manual assay) Kd @ 4°C',
 'Nematostella vectensis Kd @ 37°C',
 'Nematostella vectensis Kd @ 21°C',
 'Nematostella vectensis Kd @ 4°C',
 'Monosiga brevicollis Kd @ 37°C',
 'Monosiga brevicollis Kd @ 21°C',
 'Monosiga brevicollis Kd @ 4°C',
 'Saccharomyces cerevisiae Kd @ 37°C',
 'Saccharomyces cerevisiae Kd @ 21°C',
 'Saccharomyces cerevisiae 

In [11]:
from matplotlib.patches import Patch

In [12]:
heatmap_dir = OUTPUT_DIR / 'heatmaps'
if not heatmap_dir.exists():
    os.makedirs(heatmap_dir)
    
for name, df in tqdm(dfs_categorical.items(), total=len(dfs_categorical)):
    matrix = cluster_categorical(df)
    matrix = matrix.astype(str).replace('nan', 'N/A')

    figure = plt.figure(figsize=(0.15*len(matrix.columns) + 0.5, 0.15*len(matrix)))
    ax = plt.gca()

    catheat.heatmap(matrix,
                    cmap=palette, 
                    ax=ax, 
                    leg_pos='top',
                    linewidth=0.5,
                    legend=False,
                    leg_kws=dict(title='Kd'),
                    yticklabels=1,
                    xticklabels=1)


    legend_entries = []
    for entry in ['<1', 
                 '1-10', 
                 '10-50', 
                 '50-250', 
                 '250-1000', 
                 '1000-5000', 
                 '>5000',
                 'N/A']:

        p = Patch(facecolor=palette[entry], 
                  edgecolor='#d9d9d9',
                  label=entry)
        legend_entries.append(p)

    leg = ax.legend(handles=legend_entries, loc='center left', 
                    bbox_to_anchor=(1, 0.5),
                    title='Dissociation constant')   

    leg._legend_box.align = "left"

    ax.xaxis.set_tick_params(length=0)
    ax.yaxis.set_tick_params(length=0)

    ax.set_ylabel('Protein, acceptor fluorophore (TAMRA)')
    ax.set_xlabel('Protein, donor fluorophore (fluorescein)')

    ax.set_title(parse_name(name))

    plt.savefig(heatmap_dir / f'heatmap.{name}.pdf', bbox_inches='tight')

    plt.close()

100%|██████████| 30/30 [00:19<00:00,  1.50it/s]


In [13]:
dfs_categorical['CS_37_Kd'].shape

(58, 58)

In [14]:
df.head()

donor_protein,DDIT3,CEBPG,CEBPA,CEBPE,CREB1,CREB3L3,CREB3L1,CREBZF,XBP1,ATF6,...,CI17,CI18,CI19,CI20,CI21,CI22,CI23,CI24,DRE4A,DRE4B
acceptor_protein,,,,,,,,,,,,,,,,,,,,,
DDIT3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,50-250,10-50,10-50,1-10,>5000,>5000,>5000,>5000,<1,1-10
CEBPG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,10-50,1-10,<1,1-10,>5000,>5000,50-250,>5000,<1,<1
CEBPA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,>5000,>5000,10-50,10-50,>5000,>5000,1000-5000,250-1000,<1,<1
CEBPE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,>5000,50-250,>5000,1-10,>5000,>5000,>5000,>5000,1-10,<1
CREB1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,>5000,50-250,>5000,>5000,>5000,>5000,>5000,>5000,>5000,>5000


# Network

In [15]:
import matplotlib.colors

def color_rgb(hex_color):
    r, g, b = matplotlib.colors.hex2color(hex_color)
    
    return dict(r=int(r*255), g=int(g*255), b=int(b*255), a=1.0)

In [16]:
import itertools
import networkx as nx


In [17]:
network_dir = OUTPUT_DIR / 'networks'
if not network_dir.exists():
    os.makedirs(network_dir)

order = dict(zip(df.iloc[0].cat.categories, range(len(df.iloc[0].cat.categories))))

thicknesses = {'<1': 2,
              '1-10': 1.5,
              '10-50': 1.0,
              '50-250': 0.5,
              '250-1000': 0.1,
              '1000-5000': 0.1,
              '>5000': 0.1}


weights = {'<1': 7,
          '1-10': 6,
          '10-50': 5,
          '50-250': 4,
          '250-1000': 3,
          '1000-5000': 2,
          '>5000': 1}

network_palette = {
    '<1': palettable.cartocolors.sequential.agSunset_7.hex_colors[0],
    '1-10': palettable.cartocolors.sequential.agSunset_7.hex_colors[1],
    '10-50': palettable.cartocolors.sequential.agSunset_7.hex_colors[2],
    '50-250': palettable.cartocolors.sequential.agSunset_7.hex_colors[3],
    '250-1000': palettable.cartocolors.sequential.agSunset_7.hex_colors[4],
    '1000-5000': palettable.cartocolors.sequential.agSunset_7.hex_colors[5],
    '>5000': palettable.cartocolors.sequential.agSunset_7.hex_colors[6]
}

# Edges won't be drawn for anything above this:
max_kd = '50-250'

for name in dfs_categorical:
    df = dfs_categorical[name]
 
    proteins = set(df.columns) | set(df.index)


    graph = nx.Graph()
    graph.add_nodes_from(proteins)

    for a, b in itertools.combinations(proteins, 2):

        # Get maximum value between ab and ba

        ab = df.loc[a,b]
        ba = df.loc[b,a]

        max_value = ab
        if pd.isnull(ab):
            max_value = ba
        elif not pd.isnull(ba) and order[ba] > order[ab]:
            max_value = ba

        if not pd.isnull(max_value) and order[max_value] <= order[max_kd]:
            thick = thicknesses[max_value]
            weight = weights[max_value]
            color = color_rgb(network_palette[max_value])
            
            graph.add_edge(a, b, 
                           weight=weight,
                           viz=dict(thickness=thick, shape='solid', color=color),
                           kd=max_value)
    
    
    pos = nx.spring_layout(graph, k=1.5)
    
    vizs = {}
    
    for node in graph.nodes:
        
        color = color_rgb(network_palette['<1'])
        
        x, y = pos[node]
        
        vizs[node] = dict(color=color, pos=dict(x=x, y=y, z=1.0), size=30)
    
    nx.set_node_attributes(graph, vizs, name='viz')
        

    nx.write_gexf(graph, network_dir / f'network.{name}.gexf')
